In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
%%writefile chat_bot_pre_train_mod.py
import streamlit as st
from unsloth import FastLanguageModel
import torch
from unsloth.chat_templates import get_chat_template
from transformers import pipeline

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

fourbit_models = ["unsloth/Phi-3.5-mini-instruct"]
def create_response(choose_model_name):
  if choose_model_name == "Pre-trained phi-3":
    model_name = 'unsloth/Phi-3.5-mini-instruct'
  else:
    model_name = 'lora_model'
  model, tokenizer = FastLanguageModel.from_pretrained(
  model_name = model_name,
  max_seq_length = max_seq_length,
  dtype = dtype,
  load_in_4bit = load_in_4bit)

  tokenizer = get_chat_template(tokenizer,
  chat_template="phi-3",  # Use phi-3 template for this model
  mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"})

  FastLanguageModel.for_inference(model)
  return model, tokenizer




st.title("Chatbot")
st.sidebar.title("Choose a Model")
model_choice = st.sidebar.selectbox("Select a Model", ("Pre-trained phi-3", "Fituned Phi-3"))
model, tokenizer = create_response(model_choice)
pipe = pipeline('text-generation', model = model, tokenizer = tokenizer)
# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display previous messages from the chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Capture user input
prompt = st.chat_input("Enter your message:")

if prompt:
    # Display the user's message in the chat UI
    with st.chat_message("user"):
        st.markdown(prompt)

    # Add the user's message to the session state history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Prepare the input for generation using Unsloth's tokenizer and chat template
    messages = [{"from": "human", "value": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")  # Send input to GPU if available

    coded_input = tokenizer.batch_decode(inputs, skip_special_tokens=True)[0]
    generation_args = {"return_full_text": False, "do_sample": False}
    #generation_args = {"max_new_tokens": 128}
    response = pipe(coded_input, **generation_args)[0]['generated_text']

    with st.chat_message("assistant"):
        st.markdown(response)

    # Add the assistant's response to the session state history
    st.session_state.messages.append({"role": "assistant", "content": response})

Overwriting chat_bot_pre_train_mod.py


In [ ]:
!npm install -g localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.125.204.207


In [ ]:
!streamlit run chat_bot_pre_train_mod.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.204.207:8501

your url is: https://sour-groups-open.loca.lt
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2024-09-30 16:07:13.723741: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 16:07:13.766138: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 16:07:13.778905: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 16:07:13.809792: I tensorflow/core/platform/cpu_feature_gu